In [11]:
import nltk
nltk.download('vader_lexicon')
import pandas as pd


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/apple2015/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [12]:
import dask.dataframe as dd

In [24]:
# Load your dataset
df = dd.read_csv('data/quotes.csv')

Processing data in Batches

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499709 entries, 0 to 499708
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   quote     499708 non-null  object
 1   author    497956 non-null  object
 2   category  499646 non-null  object
dtypes: object(3)
memory usage: 11.4+ MB


In [9]:
#function to add sentiment
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sia =  SentimentIntensityAnalyzer()

#function to get sentiment labels
def get_sentiment_label(quote_v):
    sentiment = sia.polarity_scores(quote_v)
    compound = sentiment['compound']
    if compound >= 0.05:
        return "Positive"
    elif compound <= -0.05:
        return "Negative"
    else:
        return "Neutral"




In [10]:
batch_size = 10000
results = []
#
# for start in range(0, len(df), batch_size):
#     batch = df.iloc[start:start + batch_size]
#     batch['sentiment'] = batch['quote'].apply(get_sentiment_label)
#     results.append(batch)
#
#
# # Combine batches
# df = pd.concat(results, ignore_index=True)
#
# # Save results to a file
# df.to_csv('processed_dataset.csv', index=False)

/var/folders/vq/2z3hvmh94k13pb_1b0fmr7wm0000gn/T/ipykernel_40491/3352971751.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch['sentiment'] = batch['quote'].apply(get_sentiment_label)
/var/folders/vq/2z3hvmh94k13pb_1b0fmr7wm0000gn/T/ipykernel_40491/3352971751.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch['sentiment'] = batch['quote'].apply(get_sentiment_label)
/var/folders/vq/2z3hvmh94k13pb_1b0fmr7wm0000gn/T/ipykernel_40491/3352971751.py:6: SettingWithCopyWarning: 
A value is trying to be

AttributeError: 'float' object has no attribute 'encode'

In [16]:
# Ensure all quotes are strings
df['quote'] = df['quote'].astype(str)

In [17]:
# Fill missing values with an empty string
df['quote'] = df['quote'].fillna('')


In [18]:
# Load dataset into Dask
# df = dd.read_csv('large_dataset.csv')  # Replace with your file path

# Apply function to Dask DataFrame
df['sentiment'] = df['quote'].map(get_sentiment_label, meta=('quote', 'str'))

# Compute the final DataFrame
df = df.compute()

# Save results
df.to_csv('processed_dataset.csv', index=False)

AttributeError: 'float' object has no attribute 'encode'

In [20]:
import pandas as pd


# Initialize the Vader Sentiment Analyzer
sia = SentimentIntensityAnalyzer()

# Function to get sentiment labels
def get_sentiment_label(quote_v):
    if not isinstance(quote_v, str):  # Handle non-string inputs
        return "Neutral"
    sentiment = sia.polarity_scores(quote_v)
    compound = sentiment['compound']
    if compound >= 0.05:
        return "Positive"
    elif compound <= -0.05:
        return "Negative"
    else:
        return "Neutral"

# Load a sample of 30k rows from the dataset
df = pd.read_csv('data/quotes.csv', nrows=50000)  # Only process the first 30,000 rows

# Ensure the column with quotes is string and handle missing values
df['quote'] = df['quote'].fillna('').astype(str)

# Apply the sentiment function to add a sentiment column
df['sentiment'] = df['quote'].apply(get_sentiment_label)

# Balance the dataset by selecting an equal number of each sentiment
sample_size = 20000  # Number of rows to sample per sentiment
positive = df[df['sentiment'] == 'Positive'].sample(n=sample_size, random_state=42, replace=True)
negative = df[df['sentiment'] == 'Negative'].sample(n=sample_size, random_state=42, replace=True)
neutral = df[df['sentiment'] == 'Neutral'].sample(n=sample_size, random_state=42, replace=True)

# Combine the sampled data into one DataFrame
balanced_data = pd.concat([positive, negative, neutral])

# Shuffle the data
balanced_data = balanced_data.sample(frac=1, random_state=42)

# Export the balanced dataset to a CSV file
balanced_data.to_csv('balanced_quotes.csv', index=False)

print("Balanced dataset with an equal number of sentiments has been saved to 'balanced_quotes.csv'.")


Balanced dataset with an equal number of sentiments has been saved to 'balanced_quotes.csv'.


In [21]:

balanced_quotes = pd.read_csv('balanced_quotes.csv')

balanced_quotes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   quote      60000 non-null  object
 1   author     59982 non-null  object
 2   category   59993 non-null  object
 3   sentiment  60000 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


In [22]:
balanced_quotes.describe()

,quote,author,category,sentiment
count,60000,59982,59993,60000
unique,31828,13775,22928,3
top,A story is a letter that the author writes to ...,Debasish Mridha,writing,Positive
freq,10,1159,1008,20000


In [23]:

df = pd.read_csv('data/quotes.csv', nrows=60000)  # Only process the first 60,000 rows
# Shuffle the data
balanced_data = balanced_data.sample(frac=1, random_state=42)

# Export the balanced dataset to a CSV file
balanced_data.to_csv('data/balanced_quotes.csv', index=False)